In [3]:
import numpy   as np
import pandas  as pd
import seaborn as sb

In [63]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

from pyspark.sql.functions import *
from pyspark.sql.types     import *

In [6]:
spark = SparkSession\
        .builder\
        .appName('w261-final')\
        .master('local[*]')\
        .getOrCreate()

sc = spark.sparkContext
sq = SQLContext(sc)

In [10]:
!ls data

dac.tar  readme.txt  test.txt  train.txt


In [37]:
schema = StructType()

for f in range(0, 39 + 1):
    if   f == 0 : schema.add(StructField(f'CTR',        IntegerType(), True))
    elif f < 14 : schema.add(StructField(f'I{f   :02}', IntegerType(), True))
    else        : schema.add(StructField(f'C{f-13:02}',  StringType(), True))


In [38]:
train  = sq.read.format('csv')\
                .options(header = 'true', inferSchema = 'true', delimiter = '\t')\
                .schema(schema)\
                .load('data/train.txt') 

In [55]:
test   = sq.read.format('csv')\
                .options(header = 'true', inferSchema = 'true', delimiter = '\t')\
                .schema(schema[1:])\
                .load('data/test.txt') 

In [56]:
test

DataFrame[I01: int, I02: int, I03: int, I04: int, I05: int, I06: int, I07: int, I08: int, I09: int, I10: int, I11: int, I12: int, I13: int, C01: string, C02: string, C03: string, C04: string, C05: string, C06: string, C07: string, C08: string, C09: string, C10: string, C11: string, C12: string, C13: string, C14: string, C15: string, C16: string, C17: string, C18: string, C19: string, C20: string, C21: string, C22: string, C23: string, C24: string, C25: string, C26: string]

In [57]:
train_total = train.count()
test_total  = test.count()

In [43]:
train[train.CTR == 1].count()/train_total

0.25622338931920113

In [69]:
train.agg(*[(count(c)/train_total).alias(c) for c in train.columns]).toPandas().T

,0
CTR,1.000000
I01,0.546394
I02,1.000000
I03,0.785355
I04,0.783219
I05,0.974191
I06,0.776348
I07,0.956744
I08,0.999503
I09,0.956744


In [70]:
train[[c for c in train.columns if 'I' in c or 'CTR' in c]].describe().toPandas().T

,0,1,2,3,4
summary,count,mean,stddev,min,max
CTR,45840616,0.25622338931920113,0.4365466392520697,0,1
I01,25047060,3.5024134169838694,9.429076582074764,0,5775
I02,45840616,105.84842208490392,391.4578266505335,-3,257675
I03,36001169,26.913041629287093,397.9725885331938,0,65535
I04,35903247,7.322680452829238,8.79323075017961,0,969
I05,44657499,18538.99204906213,69394.60257569488,0,23159456
I06,35588288,116.06185400095671,382.566454284951,0,431037
I07,43857750,16.33313006253171,66.04975599781022,0,56311
I08,45817843,12.517042367097028,16.68888467758354,0,6047
